In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip install evaluate
!pip install rouge_score
!pip install tensorboard
!pip install gdown

import os
os.environ["WANDB_DISABLED"] = "true"
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-3B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2024.11.11: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none", 
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False, 
    loftq_config = None,
)

Unsloth 2024.11.11 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

<a name="Data"></a>
### Data Prep


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

!gdown 1dBLjJhIV8olnOLLHnS_j0wFaPc1CzuZP
!gdown 1NLF9FeN3r9NkiNHsR2b1pweczlDw37kx
!gdown 1CJ6F9LUcusyEKxihsjj3TqL2ofMwT7J5

# Load the Excel file
train_file_path = "/kaggle/working/train_dataset.csv"
test_file_path = "/kaggle/working/test_dataset.csv"
val_file_path = "/kaggle/working/val_dataset.csv"

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
val_df = pd.read_csv(val_file_path)

# Display the first few rows of the train and test sets to ensure everything works fine
print(train_df.head())

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From: https://drive.google.com/uc?id=1dBLjJhIV8olnOLLHnS_j0wFaPc1CzuZP
To: /kaggle/working/test_dataset.csv
100%|█████████████████████████████████████████| 389k/389k [00:00<00:00, 102MB/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From: https://drive.google.com/uc?id=1NLF9FeN3r9NkiNHsR2b1pweczlDw37kx
To: /kaggle/working/train_dataset.csv
100%|███████████████████████████████████████| 1.56M/1.56M [00:00<00:00, 162MB/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From: https://drive.google.com/uc?id=1CJ6F9LUcusyEKxihsjj3TqL2ofMwT7J5
To: /kaggle/working/val_dataset.csv
100%|██████████████████████████████████████| 80.1k/80.1k [00:00<00:00, 71.5MB/s]
                                                Code  \
0  @Nonnull\n        public Modal build()\n      ...   
1  public static Snowflake of(final Instant times...   
2                    boolean isMentionRepliedUser();   
3  public UnknownChannel(final GatewayDiscordClie...   
4  public int getTypeRaw()\n    {\n        return...   

                                             JavaDoc  \
0  /**\n         * Builds and returns the {@link ...   
1  /**\n     * Constructs a {@code Snowflake} uti...   
2  /**\n     * Whether this message would mention...   
3  /**\n     * Constructs an {@code UnknownChanne...   
4  /**\n     * The raw type value used to derive ...   

                                   Package  
0  net.dv8tion.jda.api.interactions.modals  
1                    discord4j.co

In [5]:
llm_few_shot_examples = """
Here are a few examples

@Nonnull
public Modal build() {
    Checks.check(!components.isEmpty(), "Cannot make a modal without components!");
    Checks.check(components.size() <= MAX_COMPONENTS, "Cannot make a modal with more than 5 components!");

    return new ModalImpl(id, title, components);
}

net.dv8tion.jda.api.interactions.modals

/**
 * Builds and returns the {@link Modal}.
 *
 * @throws IllegalArgumentException
 *         <ul>
 *             <li>If no components are added</li>
 *             <li>If more than {@value MAX_COMPONENTS} component layouts are added</li>
 *         </ul>
 *
 * @return A Modal
 */

@Nonnull
@CheckReturnValue
PermissionOverrideAction upsertPermissionOverride(@Nonnull IPermissionHolder permissionHolder);

net.dv8tion.jda.api.entities.channel.attribute

/**
 * Creates a new override or updates an existing one.
 * <br>This is similar to calling {@link PermissionOverride#getManager()} if an override exists.
 *
 * @param  permissionHolder
 *         The Member/Role for the override.
 *
 * @throws net.dv8tion.jda.api.exceptions.InsufficientPermissionException
 *         If we don't have the permission to {@link net.dv8tion.jda.api.Permission#MANAGE_PERMISSIONS MANAGE_PERMISSIONS}.
 * @throws java.lang.IllegalArgumentException
 *         If the provided permission holder is null or not from this guild.
 *
 * @return {@link net.dv8tion.jda.api.requests.restaction.PermissionOverrideAction}
 *         <br>With the current settings of an existing override or a fresh override with no permissions set.
 *
 * @see    PermissionOverrideAction#clear(long)
 * @see    PermissionOverrideAction#grant(long)
 * @see    PermissionOverrideAction#deny(long)
 */

public LegacyEmbedCreateSpec setFooter(String text, @Nullable String iconUrl) {
    requestBuilder.footer(EmbedFooterData.builder()
        .text(text)
        .iconUrl(iconUrl == null ? Possible.absent() : Possible.of(iconUrl))
        .build());
    return this;
}

discord4j.core.spec.legacy

/**
 * Sets the footer of the embed.
 *
 * @param text The footer text.
 * @param iconUrl An icon URL to display in the footer.
 * @return This spec.
 */
"""


In [6]:
llm_one_shot_example = """
Here are an example

@Nonnull
public Modal build() {
    Checks.check(!components.isEmpty(), "Cannot make a modal without components!");
    Checks.check(components.size() <= MAX_COMPONENTS, "Cannot make a modal with more than 5 components!");

    return new ModalImpl(id, title, components);
}

net.dv8tion.jda.api.interactions.modals

/**
 * Builds and returns the {@link Modal}.
 *
 * @throws IllegalArgumentException
 *         <ul>
 *             <li>If no components are added</li>
 *             <li>If more than {@value MAX_COMPONENTS} component layouts are added</li>
 *         </ul>
 *
 * @return A Modal
 */
"""

In [7]:
llm_prompt = """Given a function as Code and package of that function as Package, the task is to generate javadoc for the function.

### Code:
{}

### Package:
{}

### JavaDoc:
{}"""



EOS_TOKEN = "</eos>" # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    codes = examples["Code"]
    packages = examples["Package"]
    javadocs = examples["JavaDoc"]
    texts = []
    for code, package, javadoc in zip(codes, packages, javadocs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = llm_prompt.format(code, package, javadoc) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def formatting_few_shot_prompts_func(examples):
    codes = examples["Code"]
    packages = examples["Package"]
    javadocs = examples["JavaDoc"]
    texts = []
    for code, package, javadoc in zip(codes, packages, javadocs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = llm_prompt.format(code, package, javadoc) + "\n" +llm_few_shot_examples + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def formatting_one_shot_prompts_func(examples):
    codes = examples["Code"]
    packages = examples["Package"]
    javadocs = examples["JavaDoc"]
    texts = []
    for code, package, javadoc in zip(codes, packages, javadocs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = llm_prompt.format(code, package, javadoc) + "\n" +llm_one_shot_example + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
test_dataset = test_dataset.map(formatting_prompts_func, batched=True)
test_few_shot = test_dataset.map(formatting_few_shot_prompts_func, batched=True)
test_one_shot = test_dataset.map(formatting_one_shot_prompts_func, batched=True)
val_dataset = val_dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/2777 [00:00<?, ? examples/s]

Map:   0%|          | 0/695 [00:00<?, ? examples/s]

Map:   0%|          | 0/695 [00:00<?, ? examples/s]

Map:   0%|          | 0/695 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [8]:
train_dataset[10]

{'Code': 'public Integer getAnswerId() {\n        return this.data.answerId().get(); // We can safely call get() here because the answerId is always present\n    }',
 'JavaDoc': '/**\n     * Gets the answer id of this poll answer.\n     *\n     * @return the answer id of this poll answer\n     */',
 'Package': 'discord4j.core.object.entity.poll',
 'text': 'Given a function as Code and package of that function as Package, the task is to generate javadoc for the function.\n\n### Code:\npublic Integer getAnswerId() {\n        return this.data.answerId().get(); // We can safely call get() here because the answerId is always present\n    }\n\n### Package:\ndiscord4j.core.object.entity.poll\n\n### JavaDoc:\n/**\n     * Gets the answer id of this poll answer.\n     *\n     * @return the answer id of this poll answer\n     */</eos>'}

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
from transformers import Trainer, TrainingArguments, EvalPrediction
import evaluate
import numpy as np

# Load BLEU and ROUGE metrics using the evaluate library
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids, labels

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions[0]

    # Replace -100 with the pad token ID
    pred_ids = np.where(pred_ids == -100, tokenizer.pad_token_id, pred_ids)
    labels_ids = np.where(labels_ids == -100, tokenizer.pad_token_id, labels_ids)

    # Decode the predicted and reference sequences
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Compute ROUGE scores
    rouge_output = rouge_metric.compute(
        predictions=pred_str,
        references=label_str,
        rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"],
    )

    # Compute BLEU score
    bleu_output = bleu_metric.compute(
        predictions=pred_str,
        references=[[ref] for ref in label_str]
    )

    return {
        "BLEU": round(bleu_output["bleu"], 4),
        "R1": round(rouge_output["rouge1"], 4),
        "R2": round(rouge_output["rouge2"], 4),
        "RL": round(rouge_output["rougeL"], 4),
        "RLsum": round(rouge_output["rougeLsum"], 4),
    }

In [10]:
%load_ext tensorboard
%tensorboard --logdir /content/outputs/runs

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "tensorboard",
         # Evaluation settings
        evaluation_strategy = "steps",  
        eval_steps = 5,                
        save_steps = 50,               
        load_best_model_at_end = True,  
    ),
)
# For zero shot results
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/2777 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/695 [00:00<?, ? examples/s]

{'eval_loss': 2.2422244548797607,
 'eval_model_preparation_time': 0.0091,
 'eval_BLEU': 0.3362,
 'eval_R1': 0.562,
 'eval_R2': 0.277,
 'eval_RL': 0.4627,
 'eval_RLsum': 0.5431,
 'eval_runtime': 199.9758,
 'eval_samples_per_second': 3.475,
 'eval_steps_per_second': 1.74}

In [ ]:
# For one shot results
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_one_shot, #adding one shot
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5, 
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "tensorboard",
         # Evaluation settings
        evaluation_strategy = "steps",  
        eval_steps = 5,                
        save_steps = 50,              
        load_best_model_at_end = True,  
    ),
)
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/2777 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/695 [00:00<?, ? examples/s]

{'eval_loss': 1.7619190216064453,
 'eval_model_preparation_time': 0.0101,
 'eval_BLEU': 0.4101,
 'eval_R1': 0.6457,
 'eval_R2': 0.3243,
 'eval_RL': 0.5263,
 'eval_RLsum': 0.6294,
 'eval_runtime': 308.6297,
 'eval_samples_per_second': 2.252,
 'eval_steps_per_second': 1.128}

In [ ]:
# For few shot results
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_few_shot,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5, 
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "tensorboard",
         # Evaluation settings
        evaluation_strategy = "steps",  
        eval_steps = 5,                
        save_steps = 50,               
        load_best_model_at_end = True,  # Load the best model at the end of training
    ),
)
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/2777 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/695 [00:00<?, ? examples/s]

{'eval_loss': 1.4268790483474731,
 'eval_model_preparation_time': 0.0096,
 'eval_BLEU': 0.4743,
 'eval_R1': 0.6852,
 'eval_R2': 0.3774,
 'eval_RL': 0.5708,
 'eval_RLsum': 0.6783,
 'eval_runtime': 599.6152,
 'eval_samples_per_second': 1.159,
 'eval_steps_per_second': 0.58}

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset, #setting up the val dataset here
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5, 
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "tensorboard",
         # Evaluation settings
        evaluation_strategy = "steps",  
        eval_steps = 20,                # Evaluate every 20 steps
        save_steps = 60,               # Save the model every 60 steps
        load_best_model_at_end = True,  # Load the best model at the end of training
    ),
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/2777 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/139 [00:00<?, ? examples/s]

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,777 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 435
 "-____-"     Number of trainable parameters = 29,933,568


Step,Training Loss,Validation Loss,Bleu,R1,R2,Rl,Rlsum
20,0.873100,0.852805,0.491500,0.714800,0.535900,0.668300,0.709700
40,0.733700,0.758274,0.494600,0.736300,0.563100,0.691600,0.732000
60,0.706100,0.693258,0.502800,0.749500,0.585400,0.708100,0.744900
80,0.662500,0.639557,0.517800,0.759800,0.601800,0.722600,0.755500
100,0.601300,0.593062,0.533300,0.769600,0.626400,0.737300,0.766300
120,0.638400,0.564764,0.539700,0.776700,0.638400,0.748000,0.774600
140,0.565600,0.541105,0.555000,0.777700,0.644900,0.749600,0.775400
160,0.561100,0.519200,0.565800,0.785300,0.655900,0.758300,0.783000
180,0.468700,0.507794,0.568700,0.788200,0.659200,0.759600,0.786100
200,0.495800,0.499548,0.568400,0.783600,0.660300,0.758800,0.781500


In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset, 
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5, 
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "tensorboard",
         # Evaluation settings
        evaluation_strategy = "steps", 
        eval_steps = 5,                
        save_steps = 50,               
        load_best_model_at_end = True,  # Load the best model at the end of training
    ),
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/2777 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/695 [00:00<?, ? examples/s]

In [18]:
#For fine tuned results
trainer.evaluate()

{'eval_loss': 0.5060594081878662,
 'eval_model_preparation_time': 0.0104,
 'eval_BLEU': 0.5763,
 'eval_R1': 0.7936,
 'eval_R2': 0.6737,
 'eval_RL': 0.7676,
 'eval_RLsum': 0.7908,
 'eval_runtime': 195.9505,
 'eval_samples_per_second': 3.547,
 'eval_steps_per_second': 1.776}